# K-Fold

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from tensorflow.keras.utils import to_categorical # one-hot keras encording

import numpy as np
import pandas as pd
import tensorflow as tf

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

## 1. 데이터 읽어오기

In [4]:
df = pd.read_csv('C:/Users/user/ILIFO/DeepLearning/deeplearning_class-master/dataset/sonar.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


## 2. 데이터 전처리

In [5]:
dataset = df.values
dataset

array([[0.02, 0.0371, 0.0428, ..., 0.009, 0.0032, 'R'],
       [0.0453, 0.0523, 0.0843, ..., 0.0052, 0.0044, 'R'],
       [0.0262, 0.0582, 0.1099, ..., 0.0095, 0.0078, 'R'],
       ...,
       [0.0522, 0.0437, 0.018, ..., 0.0077, 0.0031, 'M'],
       [0.0303, 0.0353, 0.049, ..., 0.0036, 0.0048, 'M'],
       [0.026, 0.0363, 0.0136, ..., 0.0061, 0.0115, 'M']], dtype=object)

In [7]:
X = dataset[:,0:60]
X

array([[0.02, 0.0371, 0.0428, ..., 0.0084, 0.009, 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018, ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049, ..., 0.0079, 0.0036, 0.0048],
       [0.026, 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]], dtype=object)

In [51]:
X = X.astype(float)
Y_obj = dataset[:,60]

In [28]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

In [46]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
Y_encoded = tf.keras.utils.to_categorical(Y)
type(Y_encoded)

numpy.ndarray

In [16]:
# 10개의 파일로 쪼갬
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [52]:
#데이터셋 분리 (Train & Test)
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y_encoded, test_size = 0.3)

In [53]:
x_train

array([[0.0211, 0.0128, 0.0015, ..., 0.0039, 0.0051, 0.0015],
       [0.0762, 0.0666, 0.0481, ..., 0.0048, 0.0107, 0.0094],
       [0.0187, 0.0346, 0.0168, ..., 0.0115, 0.0193, 0.0157],
       ...,
       [0.0093, 0.0269, 0.0217, ..., 0.0116, 0.006 , 0.011 ],
       [0.0366, 0.0421, 0.0504, ..., 0.0017, 0.0027, 0.0027],
       [0.0158, 0.0239, 0.015 , ..., 0.0046, 0.0022, 0.0021]])

## 3. 모델 설계

In [47]:
type(y_train)

numpy.ndarray

In [55]:
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2,  activation='sigmoid'))

## 4. 모델 컴파일

In [56]:
model.compile(loss = 'binary_crossentropy', optimizer = "Adam" ,metrics = ['accuracy'])  #optimizer = "adam" 으로 설정하면 xavier 초기화

## 5. 학습

In [57]:
h= model.fit(x_train,y_train,epochs=130,batch_size=5)

Epoch 1/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6938 - accuracy: 0.5655
Epoch 2/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6754 - accuracy: 0.6345
Epoch 3/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6629 - accuracy: 0.6414
Epoch 4/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.6828
Epoch 5/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6335 - accuracy: 0.6897
Epoch 6/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6179 - accuracy: 0.7310
Epoch 7/130
29/29 [==============================] - 0s 2ms/step - loss: 0.6002 - accuracy: 0.7448
Epoch 8/130
29/29 [==============================] - 0s 2ms/step - loss: 0.5817 - accuracy: 0.7655
Epoch 9/130
29/29 [==============================] - 0s 2ms/step - loss: 0.5618 - accuracy: 0.7724
Epoch 10/130
29/29 [==============================] - 0s 1ms/step - loss: 0.5467 - accuracy: 0.8069
Epoch 11/

## 6. 검증

In [ ]:
# 빈 accuracy 배열
accuracy = []

# for train, test in skf.split(X, Y):
#     print(Y[train].size, Y[test].size)
    
# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)
